# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pickle
import numpy as np
import pandas as pd
import warnings

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\32214\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\32214\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\32214\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql("SELECT * FROM MessageCats", engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
   
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# perform train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# train pipleline
pipeline.fit(X_train,Y_train)
print('Pipeline Train score: {}\n'.format(pipeline.score(X_train,Y_train)))

Pipeline Train score: 0.9965924117587224



### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pipeline.predict(X_test)

print('Pipeline Test score: {}'.format(pipeline.score(X_test,Y_test)))
print('Pipeline Overall Accuracy: {}\n'.format((Y_pred == Y_test).mean().mean()))

warnings.filterwarnings('ignore')
categories = Y.columns
Y_pred2 = pd.DataFrame(Y_pred, columns = categories)
for category in categories:
    print(category)    
    #---------------------------------------------------------------------------
    # classification_report for scikit-learn version 0.19 and older
    # does not support zero_division and output_dict parameters
    # Print full classification report
    # --------------------------------------------------------------------------
    print(classification_report(Y_test[category], Y_pred2[category]))
    
    #---------------------------------------------------------------------------
    # classification_report for scikit-learn version 0.20+
    # Print only weighted average of f1, precision, and recall for each category
    # --------------------------------------------------------------------------
    #categoryDF = pd.DataFrame(classification_report(Y_test[category], Y_pred2[category], zero_division=0,output_dict=True))
    #print(category)
    #print('    Accuracy: {}%  Precision: {}%  Recall: {}%\n'.format(
    #    categoryDF.loc['f1-score','weighted avg'].round(4),
    #    categoryDF.loc['precision','weighted avg'].round(4),
    #    categoryDF.loc['recall','weighted avg'].round(4))
    #     )

Pipeline Test score: 0.24580408910588952
Pipeline Overall Accuracy: 0.946847557047435

related
              precision    recall  f1-score   support

           0       0.72      0.28      0.41      1513
           1       0.81      0.96      0.88      4988
           2       0.54      0.36      0.43        53

    accuracy                           0.80      6554
   macro avg       0.69      0.54      0.57      6554
weighted avg       0.79      0.80      0.77      6554

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5485
           1       0.88      0.45      0.59      1069

    accuracy                           0.90      6554
   macro avg       0.89      0.72      0.77      6554
weighted avg       0.90      0.90      0.89      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy       

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x000002AB80E490D8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [10]:
parameters = {
    #'vect__max_df': (0.5, 0.75, 1.0),    
    'vect__min_df': (0.01, 0.1, 1.0),
    #'vect__max_features': (None, 5000, 10000),
    #'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),
    #'tfidf__use_idf': (True, False),
    #'clf__estimator__n_estimators': [50, 100, 200],
    #'clf__estimator__min_samples_split': [2, 3, 4],
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [11]:
from sklearn.model_selection import RandomizedSearchCV
rand = RandomizedSearchCV(pipeline, parameters, cv=5, n_iter = 5, random_state=5)
rand.fit(X_train, Y_train)
print("\nBest Score:", rand.best_score_)
print("\nBest Parameters:", rand.best_params_)
print("\nBest Estimator:", rand.best_estimator_)

C:\Users\32214\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.

  FitFailedWarning)



Best Score: 0.24407439825624483

Best Parameters: {'vect__ngram_range': (1, 3), 'vect__min_df': 0.01}

Best Estimator: Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=0.01,
                                 ngram_range=(1, 3), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokeniz...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                      

In [12]:
cv.fit(X_train, Y_train)
print("\nBest Score:", cv.best_score_)
print("\nBest Parameters:", cv.best_params_)
print("\nBest Estimator:", cv.best_estimator_)

C:\Users\32214\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.

  FitFailedWarning)



Best Score: 0.24448171677221125

Best Parameters: {'vect__min_df': 0.01, 'vect__ngram_range': (1, 1)}

Best Estimator: Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=0.01,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokeniz...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                      

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
# to tune pipeline with best parameters, set the parameters prior to fit
params_custom = {'clf__estimator__min_samples_split': 2, 
                 'tfidf__use_idf': True, 'vect__max_df': 0.75, 
                 'vect__min_df': 0.01, 'vect__ngram_range': (1, 1)}

pipeline.set_params(**params_custom)

pipeline.fit(X_train,Y_train)
print('Tuned Pipeline Train score: {}\n'.format(pipeline.score(X_train,Y_train)))
Y_pred = pipeline.predict(X_test)
print('Tuned Pipeline Test score: {}\n'.format(pipeline.score(X_test,Y_test)))
print('Tuned Pipeline Accuracy: {}\n'.format((Y_pred == Y_test).mean().mean()))

Tuned Pipeline Train score: 0.986929101820771

Tuned Pipeline Test score: 0.2580103753433018

Tuned Pipeline Accuracy: 0.9476994541077546



In [15]:
Y_pred_2 = pd.DataFrame(Y_pred, columns = categories)
for category in categories:
    categoryDF = pd.DataFrame(classification_report(Y_test[category], Y_pred_2[category], zero_division=0,output_dict=True))
    print(category)
    print('    Accuracy: {}%  Precision: {}%  Recall: {}%\n'.format(
        categoryDF.loc['f1-score','weighted avg'].round(4),
        categoryDF.loc['precision','weighted avg'].round(4),
        categoryDF.loc['recall','weighted avg'].round(4))
         )

related
    Accuracy: 0.7672%  Precision: 0.7785%  Recall: 0.7955%

request
    Accuracy: 0.8799%  Precision: 0.8878%  Recall: 0.892%

offer
    Accuracy: 0.9927%  Precision: 0.9903%  Recall: 0.9951%

aid_related
    Accuracy: 0.755%  Precision: 0.7577%  Recall: 0.7589%

medical_help
    Accuracy: 0.8996%  Precision: 0.9084%  Recall: 0.9258%

medical_products
    Accuracy: 0.9302%  Precision: 0.926%  Recall: 0.9507%

search_and_rescue
    Accuracy: 0.9594%  Precision: 0.9498%  Recall: 0.9719%

security
    Accuracy: 0.9765%  Precision: 0.9721%  Recall: 0.9838%

military
    Accuracy: 0.9533%  Precision: 0.9384%  Recall: 0.9686%

child_alone
    Accuracy: 1.0%  Precision: 1.0%  Recall: 1.0%

water
    Accuracy: 0.9546%  Precision: 0.9586%  Recall: 0.9606%

food
    Accuracy: 0.9427%  Precision: 0.9422%  Recall: 0.9452%

shelter
    Accuracy: 0.9419%  Precision: 0.9425%  Recall: 0.9472%

clothing
    Accuracy: 0.9794%  Precision: 0.976%  Recall: 0.9858%

money
    Accuracy: 0.9658%  Prec

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class MessageLength(BaseEstimator, TransformerMixin):
    
    def message_len(self, text):
        return len(text)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.message_len)
        return pd.DataFrame(X_tagged)

def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('message_len', MessageLength())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])   

    return pipeline

model = model_pipeline()
model.fit(X_train, Y_train)
Y_model_pred = model.predict(X_test)

print('Model Train score: {}\n'.format(model.score(X_train,Y_train)))
print('Model Test score: {}\n'.format(model.score(X_test,Y_test)))
print('Model Accuracy: {}\n'.format((Y_model_pred == Y_test).mean().mean()))

Model Train score: 0.9973044451225714

Model Test score: 0.2471772963075984

Model Accuracy: 0.9466229274743159



### 9. Export your model as a pickle file

In [15]:
filename = 'MessageModel.pkl'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.